In [1]:
!git clone https://github.com/veni-jindal/FakeNewsChallenge.git

fatal: destination path 'FakeNewsChallenge' already exists and is not an empty directory.


In [2]:
# importing required packages
import pandas as pd
import numpy as np
import os
from csv import DictReader
import re
import nltk
from sklearn import feature_extraction
nltk.download('wordnet')
nltk.download('punkt')
_wnl = nltk.WordNetLemmatizer()
from tqdm import tqdm

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#!pip install --upgrade tb-nightly

In [4]:
# importing required packages
from sklearn.model_selection import train_test_split
import pandas as pd
from datetime import datetime

In [5]:
# installing bert-tensorflow
!pip install bert-tensorflow==1.0.1

In [6]:
# setting the tensorflow version 1.x
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)
import tensorflow_hub as hub

TensorFlow 1.x selected.
1.15.2


In [7]:
# importing bert 
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [8]:
# to read the stances & bodies file
class DataSet():
    def __init__(self, name, path="FakeNewsChallenge/fnc-1"):
        self.path = path

        print("Reading train dataset")
        bodies = name+"_bodies.csv"
        stances = name+"_stances.csv"
        print(stances)

        self.stances = self.read(stances)
        articles = self.read(bodies)
        self.articles = dict()

        #make the body ID an integer value
        for s in self.stances:
            s['Body ID'] = int(s['Body ID'])

        #copy all bodies into a dictionary
        for article in articles:
            self.articles[int(article['Body ID'])] = article['articleBody']

        print("Total stances: " + str(len(self.stances)))
        print("Total bodies: " + str(len(self.articles)))
    
    def read(self,filename):
        rows = []
        with open(self.path + "/" + filename, "r", encoding='utf-8') as table:
            r = DictReader(table)

            for line in r:
                rows.append(line)
        return rows

In [9]:
# getting the root word
def normalize_word(w):
    return _wnl.lemmatize(w).lower()

# tokenisation and lowering the text
def get_tokenized_lemmas(s):
    return [normalize_word(t) for t in nltk.word_tokenize(s)]

# Cleans a string: Lowercasing, trimming, removing non-alphanumeric
def clean(s):  
    return " ".join(re.findall(r'\w+', s, flags=re.UNICODE)).lower()

# Removes stopwords from a list of tokens
def remove_stopwords(l):
    return [w for w in l if w not in feature_extraction.text.ENGLISH_STOP_WORDS]


In [10]:
# getting train dataset
train = DataSet("train")
# getting test dataset
test = DataSet("competition_test")
# labels list
LABELS = ['agree', 'disagree', 'discuss', 'unrelated']
# labels - Related/Unrelated
LABELS_RELATED = ['unrelated','related']
# Getting the labels belonging to related category
RELATED = LABELS[0:3]

Reading train dataset
train_stances.csv
Total stances: 49972
Total bodies: 1683
Reading train dataset
competition_test_stances.csv
Total stances: 25413
Total bodies: 904


In [11]:
# label list
label_list = [0,1,2,3]

In [12]:
# headlines
h = []
h_test = []
# labels for stance
y = []
y_test = []
# body text
b = []
b_test = []
for s in train.stances:
  h.append(s['Headline'])
  b.append(train.articles[s['Body ID']])
  y.append(LABELS.index(s['Stance']))

for s in test.stances:
  h_test.append(s['Headline'])
  b_test.append(test.articles[s['Body ID']])
  y_test.append(LABELS.index(s['Stance']))

In [13]:
# train dataframe
train_df = pd.DataFrame(columns=['head','body', 'labels'])
train_df['head'] = h
train_df['body'] = b
train_df['labels'] = y

In [14]:
# test dataframe
test_df = pd.DataFrame(columns=['head','body', 'labels'])
test_df['head'] = h_test
test_df['body'] = b_test
test_df['labels'] = y_test

In [15]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train_df.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x['head'], 
                                                                   text_b = x['body'], 
                                                                   label = x['labels']), axis = 1)

test_InputExamples = test_df.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x['head'], 
                                                                   text_b = x['body'], 
                                                                   label = x['labels']), axis = 1)

In [16]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [17]:
# max sequence length
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 49972


INFO:tensorflow:Writing example 0 of 49972


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:label: 3 (id = 3)


INFO:tensorflow:Writing example 10000 of 25413


INFO:tensorflow:Writing example 10000 of 25413


INFO:tensorflow:Writing example 20000 of 25413


INFO:tensorflow:Writing example 20000 of 25413


In [18]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)
    
    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [19]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)
      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [20]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [21]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [22]:
OUTPUT_DIR = './output'

In [23]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [24]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': './output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f83f17ceeb8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': './output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f83f17ceeb8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [25]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [26]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving checkpoints for 4684 into ./output/model.ckpt.


INFO:tensorflow:Saving checkpoints for 4684 into ./output/model.ckpt.


INFO:tensorflow:Loss for final step: 0.0005321223.


INFO:tensorflow:Loss for final step: 0.0005321223.


Training took time  1:11:23.206364


In [28]:
# converting test data to test input features in a format which BERT accepts
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [ ]:
# predictions on test set
predictions_test = estimator.predict(test_input_fn)
# result list
res_test = []
for p in predictions_test:
  res_test.append(p['labels'])

pred_test = np.array(res_test)

In [ ]:
# predictions on test set
predictions_train = estimator.predict(train_input_fn)
# result list
res_train = []
for p in predictions_train:
  res_train.append(p['labels'])

pred_train = np.array(res_train)

In [53]:
# calculating accuracy score
def acc_score(pred, actual):
  cor = 0
  for i in range(len(pred)):
    if(pred[i] == actual[i]):
      cor+=1
  return(cor/len(pred))

In [ ]:
# accuracy and f1 score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
print("test accuracy {}".format(accuracy_score(y_true = y_test, y_pred = pred_test)))
print("test F1 score {}".format(f1_score(y_true = y_test, y_pred = pred_test, average='weighted')))

print("train accuracy {}".format(accuracy_score(y_true = y[:377472], y_pred = pred_train)))
print("train F1 score {}".format(f1_score(y_true = y_train, y_pred = pred_train, average='weighted')))

In [ ]:
# unlabeled stances and bodies
class unlab_DataSet():
    def __init__(self, name, path="FakeNewsChallenge/fnc-1"):
        self.path = path

        print("Reading test dataset")
        bodies = name+"_bodies.csv"
        stances = name+"_stances_unlabeled.csv"
        print(stances)

        self.stances = self.read(stances)
        articles = self.read(bodies)
        self.articles = dict()

        #make the body ID an integer value
        for s in self.stances:
            s['Body ID'] = int(s['Body ID'])

        #copy all bodies into a dictionary
        for article in articles:
            self.articles[int(article['Body ID'])] = article['articleBody']

        print("Total stances: " + str(len(self.stances)))
        print("Total bodies: " + str(len(self.articles)))
        
    def read(self,filename):
        rows = []
        with open(self.path + "/" + filename, "r", encoding='utf-8') as table:
            r = DictReader(table)

            for line in r:
                rows.append(line)
        return rows

In [ ]:
# load file 
competition_test = unlab_DataSet("competition_test")
unlab_h = []
unlab_b = []
for s in competition_test.stances:
  unlab_h.append(s['Headline'])
  unlab_b.append(competition_test.articles[s['Body ID']])

# dataframe
unlab_df = pd.DataFrame(columns= ['head', 'body'])
unlab_df['head'] = unlab_h
unlab_df['body'] = unlab_b
unlab_df.head()

Reading test dataset
competition_test_stances_unlabeled.csv
Total stances: 25413
Total bodies: 904


In [ ]:
# get predictions 
def getPrediction(head, body):
  input_examples = [run_classifier.InputExample(guid="", text_a = h, text_b = b, label = 0) for h,b in zip(head,body)] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(h, b, LABELS[prediction['labels']]) for h,b, prediction in zip(head,body, predictions)]

In [ ]:
# getting predictions for unlabelled data
predictions = getPrediction(unlab_h, unlab_b)

INFO:tensorflow:Writing example 0 of 25413


INFO:tensorflow:Writing example 0 of 25413


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from ./output/model.ckpt-4684


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [ ]:
# converting to numpy array
res_un = np.array(predictions)

In [ ]:
# dataframe to store predicted stances
df_res = pd.DataFrame(res_un, columns = ['Headline', 'body','Stance'])

In [ ]:
# dataframe for answe
df = pd.read_csv("FakeNewsChallenge/fnc-1/competition_test_stances_unlabeled.csv")

In [ ]:
# saving to answer.csv
df.to_csv('answer.csv', index=False, encoding='utf-8') # From pandas library